# SpaceRocks

### Vectorized coordinate transformation and ephemeris calculation with robust unit handling.

To install, simply `pip install spacerocks`

In [1]:
from spacerocks import SpaceRock
import numpy as np

You can also pass in just a single object or an array of objects. I'll try BP519. Here I've turned on precision. This involves a non-vectorized solution to Kepler's equation to calculate the eccentric anomaly, and a topocentric correction to the Earth's position. Right now, only DECam is supported, but I'm working on implementing an obscode feature.

In [2]:
BP = SpaceRock(a=4.487673062316562E+02, 
               e=9.214543710796702E-01, 
               inc=5.411068217470999E+01, 
               arg=3.480587931444684E+02, 
               node=1.352131434907198E+02, 
               epoch=2.473015776611103E+06, 
               tau=2458982.5,
               H=4.5, 
               name='BP',
               precise=True,
               input_coordinates='keplerian',
               input_frame='barycentric',
               input_angles='degrees',
               obscode=None)

In [3]:
in_data, out_data = BP.propagate(np.array([2458981.5, 2458983.5]))

In [4]:
BP.pandas_df()

,name,a,e,inc,arg,node,varpi,epoch,M,tau,...,vz,ra,dec,delta,ltt,phase_angle,elong,r,H,mag
0,BP,448.767310,0.921454,0.94441,6.074772,2.359915,2.151501,2.473016e+06,6.257774,2458981.5,...,0.001769,1.191740,-0.546574,52.016791,0.000823,0.015611,0.921327,51.413602,4.5,21.636108
1,BP,448.767303,0.921454,0.94441,6.074772,2.359915,2.151501,2.473016e+06,6.257777,2458983.5,...,0.001770,1.192571,-0.546309,52.015771,0.000822,0.015579,0.918266,51.409983,4.5,21.635912


In [5]:
BP.astropy_table()

name,a,e,inc,arg,node,varpi,epoch,M,tau,x,y,z,vx,vy,vz,ra,dec,delta,ltt,phase_angle,elong,r,H,mag
,AU,,rad,rad,rad,rad,,rad,d,AU,AU,AU,AU / d,AU / d,AU / d,rad,rad,AU,yr,rad,rad,AU,,
str2,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
BP,448.7673098667584,0.9214543718805092,0.9444095643643519,6.07477193334006,2.359914546608229,2.151501172768702,2473015.776619866,6.257773775967611,2458981.5,15.809048421340117,26.340028533411026,-41.226633488621005,-0.0026543119001561557,0.0008306485702418893,0.001769426786213318,1.1917402648264872,-0.5465739069061355,52.016790963408646,0.0008225158925439914,0.015610940292436336,0.921327447399613,51.41360154603323,4.5,21.63610791754458
BP,448.7673025861925,0.921454370276967,0.9444095645333433,6.074771928436547,2.359914545162052,2.1515011664190133,2473015.7766023455,6.257777396733507,2458983.5,15.80373972860536,26.34168971569345,-41.223094455275834,-0.0026543808305721753,0.0008305337074614667,0.001769606562654008,1.192571270408945,-0.546308880191513,52.015770998641685,0.0008224997643435561,0.015578678530211205,0.9182658967408692,51.40998271693896,4.5,21.63591249012105


In [5]:
BP.ra.hms, BP.dec.dms

(hms_tuple(h=array([4., 4.]), m=array([33., 33.]), s=array([ 7.60498809, 19.03213512])),
 dms_tuple(d=array([-31., -31.]), m=array([-18., -18.]), s=array([-58.96100771,  -4.29532377])))

In [ ]:
# geo    04 33 13.31    -31 18 31.2
# 304    04 33 13.30    -31 18 31.1

In [5]:
N = 1000
rocks = SpaceRock(a=np.random.uniform(30, 40, N), 
                  e=np.random.uniform(0, 0.999, N), 
                  inc=np.random.rayleigh(np.pi/6, N), 
                  arg=np.random.uniform(0, 2*np.pi, N), 
                  node=np.random.uniform(0, 2*np.pi, N), 
                  epoch=np.random.uniform(2473015.5, 2483015.5, N),
                  tau=2458982.5 + np.random.rand(N)*365,
                  H=np.random.uniform(4, 7, N), 
                  name=['rock{}'.format(idx) for idx in range(N)],
                  precise=True,
                  input_coordinates='keplerian',
                  input_frame='barycentric',
                  input_angles='degrees')



/usr/local/bin/anaconda/lib/python3.7/site-packages/astropy/units/quantity.py:481: RuntimeWarning: invalid value encountered in arccos
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


2.64 s ± 80.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
in_data, out_data = rocks.propagate(2459982.5)

In [3]:
import pandas as pd
from astropy.coordinates import earth
pd.options.mode.chained_assignment = None

observatories = pd.read_json('/Users/kjnapier/research/spacerocks/spacerocks/obscodes_extended.json').T
observatories['obscode'] = observatories.index
todrop = observatories[observatories.Latitude.isnull().values].index
observatories = observatories.drop(todrop)
observatories['Latitude'] = np.degrees(observatories.Latitude.values.astype(float))
observatories['Longitude'][observatories.Longitude.values > 180] -= 360
observatories = observatories.rename(columns={'Latitude': 'lat',
                                              'Longitude': 'lon',
                                              'Geocentric_dist': 'geocentric_dist',
                                              'Name': 'name'})
observatories = observatories.drop(columns=['cos', 'sin'])

elevations = np.zeros(len(observatories))
# This function isn't vectorized???
for idx in range(len(observatories)):
    x, y, z = earth.EarthLocation(lat=observatories.lat.values[idx],
                                  lon=observatories.lon.values[idx],
                                  height=0).value
    ellipsoid_edge = np.sqrt(x**2 + y**2 + z**2)
    elevations[idx] = 6378137 * observatories.geocentric_dist.values[idx] - ellipsoid_edge

observatories['elevation'] = elevations
observatories = observatories.drop(columns=['geocentric_dist'])
observatories = observatories[['obscode', 'lat', 'lon', 'elevation', 'name']]
#observatories['lat'] = [str(abs(ii)) + ' N' if ii > 0 else str(abs(ii)) + ' S' \
#                        for ii in observatories.lat]
#observatories['lon'] = [str(abs(ii)) + ' E' if ii > 0 else str(abs(ii)) + ' W' \
#                        for ii in observatories.lon]
observatories.to_csv('/Users/kjnapier/research/spacerocks/spacerocks/observatories.csv', index=False)
